# Canvas Batch Change Dates

This script allows you to adjust the start and end times of multiple courses at once in Canvas. At each stage you will be asked to give certain inputs and asked confirmation steps before moving forward. Please read through all of the steps and confirm that you have all of the information needed.

If any of the steps along the way fail, the script will not move forward. There should be indicators of why the failure occurred and steps to take to fix the errors.

There are only two places where you might need to change text in the 'cells' below. The API_URL below should be your institution url. Near the end there is a section 'Get Course List for Changes' - you can also change the name of the csv with the courses (although I recommend naming your file 'start_end_courses.csv' for consistency).

To Run:
Once you have confirmed all of the steps

Kernel -> Run All

## 1. Set the url for your instance of canvas

In the cell below you need to set the API_URL for your case. 
For UBC's main instance use "https://ubc.instructure.com" (note, this is the same as ubc.canvas.ca).
For test use "https://ubc.test.instructure.com". For beta use "https://ubc.beta.instructure.com".

You can copy and paste any one of the below into the cell below
"# Enter your API_URL here"

```
API_URL = "https://ubc.instructure.com" 
API_URL = "https://ubc.test.instructure.com"
API_URL = "https://ubc.beta.instructure.com"
```


In [ ]:
API_URL = "https://ubc.test.instructure.com"

## 2. Token Validation
This is the token validation stage. If you do not enter a valid token, nothing else will happen.

In [ ]:
%run -i "src/canvas_batch_change_dates.py"

## 3. Get Course List for Changes
1. You should have a csv called "courses_start_end.csv" in **src/csv/input** that has the columns "course_id", "start_date", and "end_date" with a list of courses that you want to apply changes to.1. Please ensure the dates are formatted as "YYYY-MM-DD" or "YYYY-MM-DD HH:MM" in your file. If you have used Excel, the columns should be formatted as a "Short Date" (YYYY-MM-DD) or "Long Date" (YYYY-MM-DD HH:MM). 
1. The script assumes that the dates are for Vancouver
1. The start_date will assume the time of start is 12:00am that day
1. The end_date will assume the time of end is 11:59pm that day


Once you have completed the script, **rename and move start_end_courses.csv** to complete with a useful name.


In [ ]:
# if you have a different file than courses.csv and it has course_id, you can also change
# start_end_courses.csv below to the correct file name (I do not recommend this)
import os
course_file = os.path.join(INPUT,"start_end_courses.csv")

## 4. Update Canvas Course Settings
Using the variables set up so far and the data loaded above, update each of the courses in the loaded list. This will run through the list of courses and attempt to update them. You should see indicators for success or failure for each of the courses as the script runs.

The default setting for restricting participations to course dates is **None**. This will keep whatever setting was initially established. You can change this value below to **True** or **False** 

**None** will keep settings as is. 
**True** will restrict participation to within the given dates.
**False** will allow participation outside of the given dates. 

In [ ]:
# restrict_participation_to_dates can be True, False, or None
restrict_participation_to_dates = True

## 5. Making the Changes
In this step you will be asked to confirm the csv that is being read, and to confirm the start and end date changes that will occur.

In [ ]:
NOW = datetime.datetime.now()

details = updateMultipleCourses(
          getCoursesFile(
          course_file), restrict_participation_to_dates)

print('\nTotal Courses: {}\n'.format(len(details)))
display(details)

## 6. Generate Final Output
The final output will be a csv with the changes that have taken place (if confirmed in the step above).

In [ ]:
# NOW is the time you made the update, not the time the csv is output
output_name = "{}\Canvas Course Settings.csv".format(OUTPUT)
createCSV(details, output_name)